In [12]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import sqlite3 as sql
from sqlalchemy import create_engine, MetaData, inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship
import pandas as pd

In [13]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
import csv
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [14]:
# Create Engine
### BEGIN SOLUTION
engine = create_engine("sqlite:///hospitalacquiredinfections.sqlite")
### END SOLUTION

In [15]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
# Make sure to specify types for each column

# Declare a Base object here
### BEGIN SOLUTION
Base = declarative_base()

### END SOLUTION

In [16]:
conn = engine.connect()
conn.text_factory = str

In [17]:
# Define the ORM class for `Measurements`
### BEGIN SOLUTION
class CMS(Base):
    
    __tablename__ = 'CMSHAI'
    
    PrimaryKey = Column(Integer, primary_key=True)
    FacilityID = Column(Integer)
    FacilityName = Column(String(100))
    Address = Column(String(100))
    City = Column(String(100))
    State = Column(String(100))
    ZIPCode = Column(String(100))
    CountyName = Column(String(100))
    PhoneNumber = Column(String(20))
    MeasureID = Column(String(50))
    MeasureName = Column(String(100))
    ComparedtoNational = Column(String(200))
    Score = Column(Integer)
    Footnote = Column(Integer)
    StartDate = Column(String(20))
    EndDate = Column(String(20))
                      
### END SOLUTION

In [18]:
# Use `create_all` to create the tables
### BEGIN SOLUTION
Base.metadata.create_all(engine)
### END SOLUTION

In [19]:
#Create the inspector and connect it to the engine
inspector = inspect(engine)




In [20]:
#Use inspector to print out the table column names
table_names = inspector.get_table_names()
print(table_names)

['CMSHAI']


In [21]:
#Use inspector to print column names and data types
columns = inspector.get_columns('CMSHAI')
for column in columns:
    print(column["name"], column["type"])

PrimaryKey INTEGER
FacilityID INTEGER
FacilityName VARCHAR(100)
Address VARCHAR(100)
City VARCHAR(100)
State VARCHAR(100)
ZIPCode VARCHAR(100)
CountyName VARCHAR(100)
PhoneNumber VARCHAR(20)
MeasureID VARCHAR(50)
MeasureName VARCHAR(100)
ComparedtoNational VARCHAR(200)
Score INTEGER
Footnote INTEGER
StartDate VARCHAR(20)
EndDate VARCHAR(20)


In [22]:
#Read csv
HAI = pd.read_csv(("HospitalAcquiredInfections.csv"), dtype=object, encoding = 'utf8')

In [23]:
Base.metadata.create_all(engine)

In [24]:
#Convert csv data to dictionary
HAICMS = HAI.to_dict(orient='records')
#View first row of dictionary
HAICMS[0]

{'FacilityID': '10001',
 'FacilityName': 'SOUTHEAST HEALTH MEDICAL CENTER',
 'Address': '1108 ROSS CLARK CIRCLE',
 'City': 'DOTHAN',
 'State': 'AL',
 'ZIPCode': '36301',
 'CountyName': 'Houston',
 'PhoneNumber': '(334) 793-8701',
 'MeasureID': 'HAI_1_CILOWER',
 'MeasureName': 'Central Line Associated Bloodstream Infection (ICU + select Wards): Lower Confidence Limit',
 'ComparedtoNational': 'No Different than National Benchmark',
 'Score': '0.203',
 'Footnote': nan,
 'StartDate': '7/1/2019',
 'EndDate': '12/31/2020'}

In [25]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [26]:
#Create sqlalchemy table
CMSTable = sqlalchemy.Table('CMSHAI', metadata, autoload=True)

In [27]:
#Insert dictionary data into sqlalchemy table
conn.execute(CMSTable.insert(), HAICMS)

In [28]:
#View sqlalchemy table data
conn.execute("select * from cmshai").fetchall()

[(1, 10001, 'SOUTHEAST HEALTH MEDICAL CENTER', '1108 ROSS CLARK CIRCLE', 'DOTHAN', 'AL', '36301', 'Houston', '(334) 793-8701', 'HAI_1_CILOWER', 'Central Line Associated Bloodstream Infection (ICU + select Wards): Lower Confidence Limit', 'No Different than National Benchmark', 0.203, None, '7/1/2019', '12/31/2020'),
 (2, 10001, 'SOUTHEAST HEALTH MEDICAL CENTER', '1108 ROSS CLARK CIRCLE', 'DOTHAN', 'AL', '36301', 'Houston', '(334) 793-8701', 'HAI_1_CIUPPER', 'Central Line Associated Bloodstream Infection (ICU + select Wards): Upper Confidence Limit', 'No Different than National Benchmark', 1.23, None, '7/1/2019', '12/31/2020'),
 (3, 10001, 'SOUTHEAST HEALTH MEDICAL CENTER', '1108 ROSS CLARK CIRCLE', 'DOTHAN', 'AL', '36301', 'Houston', '(334) 793-8701', 'HAI_1_DOPC', 'Central Line Associated Bloodstream Infection: Number of Device Days', 'No Different than National Benchmark', 8631, None, '7/1/2019', '12/31/2020'),
 (4, 10001, 'SOUTHEAST HEALTH MEDICAL CENTER', '1108 ROSS CLARK CIRCLE', 